<a href="https://colab.research.google.com/github/kentokura/ox_2x2_retrograde_analysis/blob/main/retrograde_analysis/retrograde_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ox_input

|      | PREVIOUS_STATES   | STATE   | NEXT_STATES              | RESULT   |
|------|-------------------|---------|--------------------------|----------|
| ____ | ['']              | ____    | ['o___']                 | nan      |
| o___ | ['____']          | o___    | ['o__x', 'o_x_', 'ox__'] | nan      |
| o__x | ['o___']          | o__x    | ['oo_x', 'o_ox']         | nan      |
| o_x_ | ['o___']          | o_x_    | ['o_xo', 'oox_']         | nan      |
| ox__ | ['o___']          | ox__    | ['oxo_', 'ox_o']         | nan      |
| oo_x | ['o__x']          | oo_x    | []                       | o_win    |
| o_ox | ['o__x']          | o_ox    | []                       | o_win    |
| o_xo | ['o_x_']          | o_xo    | []                       | o_win    |
| oox_ | ['o_x_']          | oox_    | []                       | o_win    |
| oxo_ | ['ox__']          | oxo_    | []                       | o_win    |
| ox_o | ['ox__']          | ox_o    | []                       | o_win    |

## 前処理 
辺の数列を追加


## 処理



```
Q をキューとする
dp テーブル全体を -1 に初期化する
for each 各ノード v do
    if deg[v] == 0 then
        dp[v] = 0
        v を Q に push

while Q が空ではない do
    Q からノード v を pop して取り出す
    for each v を終点とする辺 e とその始点のノード nv do
        if nv はすでに訪れたノード then
            continue
        辺 e をグラフから削除 (実装上は deg[nv] の値を減らせば OK)
        if dp[v] = 0 then
            dp[nv] = 1
            nv を Q に push
        else if dp[v] = 1 then
            if deg[nv] = 0 then (辺を削除して行ったことで出次数が 0 になったら負けノード確定です)
                dp[nv] = 0
                nv を Q に push
```

```
unsolvedDfをキューとする
inputDf["RESULT"]を -1 に初期化する
for each inputDf node do
    if node."NEXT_STATES" == [] then
        dp[i] = 0
        v を Q に push
```

グラフから葉をけずりおとしていくイメージ
- 前処理<br>
resultに"*_win"か"braw"が入力されている<br> -> それぞれのPrevious_statesに登録されているstateと一致するノードをinputDfからunsolvedDfへ移動、本体はsolvedDfへ移動.
- メイン処理


BFS
のこったunsolvedをあいこにする
後処理


## ox_output

|      | PREVIOUS_STATES   | STATE   | NEXT_STATES              | RESULT   |
|------|-------------------|---------|--------------------------|----------|
| ____ | ['']              | ____    | ['o___']                 | o_win      |
| o___ | ['____']          | o___    | ['o__x', 'o_x_', 'ox__'] | o_win      |
| o__x | ['o___']          | o__x    | ['oo_x', 'o_ox']         | o_win      |
| o_x_ | ['o___']          | o_x_    | ['o_xo', 'oox_']         | o_win      |
| ox__ | ['o___']          | ox__    | ['oxo_', 'ox_o']         | o_win      |
| oo_x | ['o__x']          | oo_x    | []                       | o_win    |
| o_ox | ['o__x']          | o_ox    | []                       | o_win    |
| o_xo | ['o_x_']          | o_xo    | []                       | o_win    |
| oox_ | ['o_x_']          | oox_    | []                       | o_win    |
| oxo_ | ['ox__']          | oxo_    | []                       | o_win    |
| ox_o | ['ox__']          | ox_o    | []                       | o_win    |

```

各ノードはまだ勝敗が一つもわかっていない
# 前処理
for each 各ノード do
    if ノード.枝の本数 == ０本 then
        ノード.勝敗 = 勝敗確認(ノード)
        キューにノードをpush
 
while キューが空ではない do
    キューからノードをpopして取り出す
    for each ノード.ひとつ前のノード do
        if ひとつ前のノード はすでに訪れたノード then
            continue
        ひとつ前のノード.辺の数 -= 1
        if ノード.勝敗 = "負け" then # このノードが手番Pの負けならば、一つ前の状態でその手をさせば勝てる
            ひとつ前のノード.勝敗 = "勝ち"
            ひとつ前のノード を キュー に push
        else if ノード.勝敗 = "勝ち" then
            if ひとつ前のノード.辺の数 = 0 then # 辺を削除して行ったことで出次数が 0 になったら負けノード確定
                ひとつ前のノード.勝敗 = 負け
                ひとつ前のノード を キュー に push

```

# インプット

In [1]:
# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

# githubリポジトリをドライブにクローンする
!git clone https://github.com/kentokura/ox_2x2_retrograde_analysis.git

Mounted at /content/drive
Cloning into 'ox_2x2_retrograde_analysis'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 41 (delta 11), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (41/41), done.


In [2]:
# csvの読み込むためのモジュール
import pandas as pd
from pandas import DataFrame
import numpy as np
from tabulate import tabulate # pandasのdfをきれいに出力するためのモジュール

In [3]:
# ox_inputの読み込み

inputDf = pd.read_csv(
    "/content/ox_2x2_retrograde_analysis/retrograde_analysis/ox_input.csv",
    index_col=0, # 最初の１行はデータ名。
    encoding="cp932" # windowsの追加文字に対応。おまじないだと思えば良い。
    )
print(tabulate(inputDf, inputDf.columns,tablefmt='github', showindex=True))


|      | PREVIOUS_STATES   | STATE   | NEXT_STATES              | RESULT   |
|------|-------------------|---------|--------------------------|----------|
| ____ | ['']              | ____    | ['o___']                 | nan      |
| o___ | ['____']          | o___    | ['o__x', 'o_x_', 'ox__'] | nan      |
| o__x | ['o___']          | o__x    | ['oo_x', 'o_ox']         | nan      |
| o_x_ | ['o___']          | o_x_    | ['o_xo', 'oox_']         | nan      |
| ox__ | ['o___']          | ox__    | ['oxo_', 'ox_o']         | nan      |
| oo_x | ['o__x']          | oo_x    | []                       | o_win    |
| o_ox | ['o__x']          | o_ox    | []                       | o_win    |
| o_xo | ['o_x_']          | o_xo    | []                       | o_win    |
| oox_ | ['o_x_']          | oox_    | []                       | o_win    |
| oxo_ | ['ox__']          | oxo_    | []                       | o_win    |
| ox_o | ['ox__']          | ox_o    | []                       | o_win    |

# 処理

In [4]:
# 抽象構文木の処理モジュール
# str化したlistをlistに戻す "[hoge, huga]" -> [hoge, huga]
import ast
workDf = inputDf

In [5]:
# 手番をつける
workDf["TURN_PLAYER"] = "unsolved"
workDf.loc[workDf["STATE"].str.count("_") % 2 == 0, "TURN_PLAYER"] = "o"
workDf.loc[workDf["STATE"].str.count("_") % 2 == 1, "TURN_PLAYER"] = "x"
# next_statesの辺の数の列をつくる
workDf["NEXT_STATES_NUM"] = workDf["NEXT_STATES"]
for index in workDf.index:
  num = len(ast.literal_eval(workDf.loc[index].NEXT_STATES_NUM))
  workDf.at[index, "NEXT_STATES_NUM"] = num
print(workDf)

     PREVIOUS_STATES STATE  ... TURN_PLAYER NEXT_STATES_NUM
____            ['']  ____  ...           o               1
o___        ['____']  o___  ...           x               3
o__x        ['o___']  o__x  ...           o               2
o_x_        ['o___']  o_x_  ...           o               2
ox__        ['o___']  ox__  ...           o               2
oo_x        ['o__x']  oo_x  ...           x               0
o_ox        ['o__x']  o_ox  ...           x               0
o_xo        ['o_x_']  o_xo  ...           x               0
oox_        ['o_x_']  oox_  ...           x               0
oxo_        ['ox__']  oxo_  ...           x               0
ox_o        ['ox__']  ox_o  ...           x               0

[11 rows x 6 columns]


In [7]:
各ノードはまだ勝敗が一つもわかっていない
# 前処理
for each 各ノード do
    if ノード.枝の本数 == ０本 then
        ノード.勝敗 = 勝敗確認(ノード)
        キューにノードをpush
 
while キューが空ではない do
    キューからノードをpopして取り出す
    for each ノード.ひとつ前のノード do
        if ひとつ前のノード はすでに訪れたノード then
            continue
        ひとつ前のノード.辺の数 -= 1
        if ノード.勝敗 = "負け" then # このノードが手番Pの負けならば、一つ前の状態でその手をさせば勝てる
            ひとつ前のノード.勝敗 = "勝ち"
            ひとつ前のノード を キュー に push
        else if ノード.勝敗 = "勝ち" then
            if ひとつ前のノード.辺の数 = 0 then # 辺を削除して行ったことで出次数が 0 になったら負けノード確定
                ひとつ前のノード.勝敗 = 負け
                ひとつ前のノード を キュー に push


'\n\n各ノードはまだ勝敗が一つもわかっていない\n# 前処理\nfor each 各ノード do\n    if ノード.枝の本数 == ０本 then\n        ノード.勝敗 = 勝敗確認(ノード)\n        キューにノードをpush\n \nwhile キューが空ではない do\n    キューからノードをpopして取り出す\n    for each ノード.ひとつ前のノード do\n        if ひとつ前のノード はすでに訪れたノード then\n            continue\n        ひとつ前のノード.辺の数 -= 1\n        if ノード.勝敗 = "負け" then # このノードが手番Pの負けならば、一つ前の状態でその手をさせば勝てる\n            ひとつ前のノード.勝敗 = "勝ち"\n            ひとつ前のノード を キュー に push\n        else if ノード.勝敗 = "勝ち" then\n            if ひとつ前のノード.辺の数 = 0 then # 辺を削除して行ったことで出次数が 0 になったら負けノード確定\n                ひとつ前のノード.勝敗 = 負け\n                ひとつ前のノード を キュー に push\n\n'

# 出力

In [4]:
# solvedDfをox_outputという名前で書き出し
solvedDf.to_csv('/content/ox_2x2_retrograde_analysis/retrograde_analysis/ox_output.csv')

In [5]:
# ox_outputの確認

solvedDf = pd.read_csv(
    "/content/ox_2x2_retrograde_analysis/retrograde_analysis/ox_output.csv",
    index_col=0, # 最初の１行はデータ名。
    encoding="cp932" # windowsの追加文字に対応。おまじないだと思えば良い。
    )
print(solvedDf)

     PREVIOUS_STATES STATE               NEXT_STATES RESULT
____            ['']  ____                  ['o___']    NaN
o___        ['____']  o___  ['o__x', 'o_x_', 'ox__']    NaN
o__x        ['o___']  o__x          ['oo_x', 'o_ox']    NaN
o_x_        ['o___']  o_x_          ['o_xo', 'oox_']    NaN
ox__        ['o___']  ox__          ['oxo_', 'ox_o']    NaN
oo_x        ['o__x']  oo_x                        []  o_win
o_ox        ['o__x']  o_ox                        []  o_win
o_xo        ['o_x_']  o_xo                        []  o_win
oox_        ['o_x_']  oox_                        []  o_win
oxo_        ['ox__']  oxo_                        []  o_win
ox_o        ['ox__']  ox_o                        []  o_win
